# Analisi dei Piloti del Mondiale di Formula 1



## Dataset

[Link del dataset](https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv)

Il dataset formula1_data.csv contiene le seguenti colonne: </br>
1. Driver: Nome del pilota.
2. Team: Nome del costruttore per il quale il pilota gareggia.
3. Race: Città in cui si è svolto il Gran Premio.
4. Country: Paese in cui si è svolto il Gran Premio.
5. Position: Numero compreso tra 0 e 8 che indica l'ordine di arrivo del pilota </br> (0 significa che il pilota non è arrivato tra i primi 8 e non ha ottenuto punti).


## Sistema di punteggio

Al termine di ogni Gran Premio, i punti vengono assegnati ai piloti in base al loro ordine di arrivo come segue: </br>

* 1° posto: 10 punti
* 2° posto: 8 punti
* 3° posto: 6 punti
* 4° posto: 5 punti
* 5° posto: 4 punti
* 6° posto: 3 punti
* 7° posto: 2 punti
* 8° posto: 1 punto
* 9° posto o oltre: 0 punti

## Obiettivi del progetto
### 1. Funzione per l'analisi delle performance individuali dei piloti

La prima funzione riceve in input il nome di un pilota e restituisce una lista contenente tre informazioni chiave:
* Il totale dei punti accumulati dal pilota durante il campionato.
* Il numero di vittorie, ovvero quante volte il pilota è arrivato primo in un Gran Premio.
* Il numero di podi, ovvero quante volte il pilota è arrivato tra i primi tre classificati.

Questa funzione sarà utile per analizzare le performance individuali dei piloti e avere una chiara visione delle loro posizioni nel corso della stagione.

### 2. Funzione per la creazione della classifica finale dei piloti
La seconda funzione genera un dizionario contenente i nomi dei piloti come chiavi e il loro punteggio totale come valori. Il dizionario viene poi utilizzato per creare una classifica generale dei piloti.

Infine, la classifica sarà salvata in un file di testo (Drivers_Standings_2008.txt) con il seguente formato:

Drivers Standings 2008 Formula 1 </br>
NomePilota1: PunteggioTotale </br>
NomePilota2: PunteggioTotale </br>

### 3. Funzione per la classifica dei costruttori

La terza funzione crea un dizionario con i nomi dei team/costruttori come chiavi e il loro punteggio totale come valori. Il punteggio di ciascun team è la somma dei punti ottenuti dai piloti che hanno gareggiato per quel costruttore.

Questa funzione utilizza i dati precedentemente generati per i piloti e calcola la classifica dei costruttori. Anche questa informazione è essenziale per avere una visione chiara delle prestazioni dei team durante l'anno.

---

# Realizzazione del Progetto

## Soluzione adottata

Il codice ha la seguente logica di base: inizialmente si raccolgono i dati dal dataset CSV, salvandoli all'interno di appositi dizionari; successivamente, si utilizzano tali dizionari per calcolare la performance individuale di un pilota inserito dall'utente, per generare la classifica finale dei piloti e salvarla su un file TXT, e per generare la classifica finale dei team/costruttori.

La raccolta dei dati è stata implementata all'interno della funzione ***collect_data***, che legge il dataset e restituisce i seguenti dizionari:

* Dizionario contenente i nomi dei piloti come chiavi, e il loro punteggio totale come valori

* Dizionario contenente i nomi dei piloti come chiavi, e il loro numero di vittorie come valori

* Dizionario contenente i nomi dei piloti come chiavi, e il loro numero di podi come valori

* Dizionario contenente i nomi dei team/costruttori come chiavi, e il loro punteggio totale come valori

Tutto ciò che segue la raccolta dei dati è, in sostanza, una consultazione dei suddetti dizionari: infatti, le tre funzioni richieste nel progetto si basano quasi interamente sui dizionari contenenti i dati precedentemente raccolti. <br>
All'interno della funzione ***main*** saranno poi chiamate le tre funzioni per poter essere eseguite.

Per chiarezza, si specifica che sono state effettuate le seguenti scelte:

* Nonostante il dataset sia privo di errori legati al valore della posizione di arrivo dei piloti, nella funzione ***collect_data*** viene eseguito un semplice controllo sulla variabile *position*, imponendo che essa sia un intero tra 0 e 8.

* La funzione ***drivers_rank*** non genera il dizionario {piloti : punteggio totale dei piloti}, bensì lo ottiene direttamente nell'argomento; procedendo in questo modo, è possibile risparmiare risorse computazionali ed evitare di collezionare i dati in modo ridondante.

* Analogamente, la funzione ***teams_rank*** non genera il dizionario {team : punteggio totale dei team}, ma lo ottiene nell'argomento.

Infine, si specifica che l'unico supporto utilizzato per la scrittura del codice è stata la piattaforma **ProfAI**.

## Codice

In [ ]:
# Import the dataset “formula1_data.csv” into my working directory
# On Colab/Jupiter, just type !wget https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv

import requests

url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv"
response = requests.get(url)

with open("formula1_data.csv", "wb") as file:
    file.write(response.content)

In [ ]:
import csv

def collect_data(filename = "formula1_data.csv"):

    """
    This function reads the CSV file and saves the necessary
    data in special dictionaries using the scoring system.
    The function is necessary in order to read the dataset
    only once, resulting in savings of computational resources.

    Keyword Argument: filename   (str)

    Returns:
        - driver_points   (dict)
        - victories       (dict)
        - podiums         (dict)
        - team_points     (dict)
    """

    driver_points = {}  # dictionary driver:points
    victories = {}      # dictionary driver:victories
    podiums = {}        # dictionary driver:podiums
    team_points = {}    # dictionary teams:points

    # Ordered tuple of possible scores
    scores_tuple = (0, 10, 8, 6, 5, 4, 3, 2, 1)

    # Opening the dataset in read-only mode within a context
    with open(filename, 'r', newline="", encoding='utf8') as csv_dataset:

        csv_reader = csv.DictReader(csv_dataset)

        for row in csv_reader:

            driver = row["Driver"]
            team = row["Team"]
            position = row["Position"]

            # Requiring the "position" parameter to be an integer between 0 and 8

            try:
                position = int(position)
                assert position in range(0,9), "La posizione di arrivo deve essere compresa tra 0 e 8"

            except ValueError:
                print("È stata riscontrata una posizione di tipo non intero!")
                return None, None, None, None

            except AssertionError as e:
                print(e)
                return None, None, None, None

            # Assigning the score based on the finishing position
            score = scores_tuple[position]

            # Filling the dictionaries related to drivers, taking care to initialize
            # the value related to the driver to zero if its driver key is absent

            if driver not in driver_points:
                driver_points[driver] = 0

            if driver not in victories:
                victories[driver] = 0

            if driver not in podiums:
                podiums[driver] = 0

            driver_points[driver] += score  # Updating the driver's score

            if position in range(1,4):  # Podium: top three positions

                if position == 1:   # Victory: first position
                    victories[driver] += 1

                podiums[driver] += 1

            # Filling the team dictionary, taking care to initialize
            # the team value to zero if its team key is absent

            if team not in team_points:
                team_points[team] = 0

            team_points[team] += score  # Updating the team's score

    return driver_points, victories, podiums, team_points


# 1° Function


def individual_performance(driver_name, driver_points, victories, podiums):

    """
    Function for analyzing individual driver performance:
    returns a list containing the driver's total score,
    number of wins, and number of podium finishes.

    Positional Arguments:
        - driver_name      (str)
        - driver_points    (dict)
        - victories        (dict)
        - podiums          (dict)

    Return: driver_stats   (list)
    """

    # Creating the individual performance list
    driver_stats = [driver_points[driver_name],
                    victories[driver_name],
                    podiums[driver_name]
                    ]

    return driver_stats


# 2° Function


def drivers_rank(driver_points):

    """
    Function for creating the final driver ranking:
    generates the ranking and saves it in a text file;
    returns a list sorted by decreasing drivers scores.

    Positional Argument: driver_points   (dict)

    Return: drivers_rank_list   (list)
    """

    # The "driver_points" parameter is the dictionary containing the
    # names of the drivers as keys and their total scores as values,
    # which is generated by calling "collect_data" in Main function.

    # Creating an initially empty list of tuples (Driver, Total score)
    drivers_rank_list = []

    for driver, points in driver_points.items():
        drivers_rank_list.append((driver, points))

    # Sorting the list in descending order by score
    drivers_rank_list.sort(key=lambda val: val[1], reverse=True)

    # Saving the ranking in the text file

    with open("Drivers_Standings_2008.txt", 'w', newline="", encoding='utf8') as txt_file:

        txt_file.write("Drivers Standings 2008 Formula 1 \n")   # Intestazione

        for driver, points in drivers_rank_list:
            txt_file.write(f"{driver}: {points}\n")

    print("\n"+ "La classifica finale dei piloti è stata stampata nel file di testo. \n")

    return drivers_rank_list


# 3° Function


def teams_rank(team_points):

    """
    Function for creating the final ranking of teams/builders:
    returns a list sorted by decreasing teams scores.

    Positional Argument: team_points   (dict)

    Return: teams_rank_list   (list)
    """

    # The "team_points" parameter is the dictionary containing
    # the names of teams as keys and their total score as values,
    # which is generated by calling "collect_data" in Main function.

    #Creating an initially empty list of tuples (Team, Total Score)
    teams_rank_list = []

    for team, points in team_points.items():
        teams_rank_list.append((team, points))

    # Sorting the list in descending order by score
    teams_rank_list.sort(key=lambda val: val[1], reverse=True)

    return teams_rank_list


In [ ]:
# MAIN FUNCTION

if __name__ == "__main__":

    # Collecting data within the appropriate dictionaries
    driver_points, victories, podiums, team_points = collect_data()

    # Checking that the data collection occurred correctly
    assert driver_points is not None, "Si è verificato un errore nella raccolta dei dati!"

    # Requesting the driver's name from the user and verifying
    # that the name is contained inside "driver_points" keys

    while True:
        driver_name = input("Di quale pilota vuoi analizzare la performance? ")
        if driver_name in driver_points:
            break
        else:
            print("\n" +
                  f"Il pilota {driver_name} non partecipa al Mondiale di Formula 1! \n" +
                  "Piloti disponibili:", ", ".join(driver_points.keys()) +
                  "\n"
                  )

    # Output of driver's individual performance
    driver_stats = individual_performance(driver_name, driver_points, victories, podiums)

    print("\n" +
    f"Performance individuale di {driver_name}: \n\n" +
    f"Totale dei punti: {driver_stats[0]} \n" +
    f"Numero di vittorie: {driver_stats[1]} \n" +
    f"Numero di podi: {driver_stats[2]}"
    )

    # Saving the final driver ranking in the text file, and obtaining "drivers_rank_list"
    drivers_rank_list = drivers_rank(driver_points)

    # Showing the final drivers ranking
    print("Classifica finale dei piloti: \n")
    for driver, points in drivers_rank_list:
        print(f"Pilota {driver}: {points} punti")

    print("")

    # Generating the final teams ranking, and obtaining "teams_rank_list"
    teams_rank_list = teams_rank(team_points)

    # Showing the final teams ranking
    print("Classifica finale dei team/costruttori: \n")
    for team, points in teams_rank_list:
        print(f"Team {team}: {points} punti")
